<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

## import

In [1]:
# Dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler

# LightGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# LogisticRegression
from sklearn.linear_model import LogisticRegression

# RandomForest
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#control
#!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
#import pandas_profiling
import seaborn as sns
#from pandas_profiling.utils.cache import cache_file

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

#何で必要なのか分かってない
%matplotlib inline
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 235kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 18.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 133kB 30.3MB/s 
     |████████████████████████████████| 112kB 33.0MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=2819749ef5b13fe31f1c0c65794dfdb88688d5cef40182a983d9449d745990ef
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312964 sha256=436230d06b174799d73f3db6db98ba292b6a443da9081b27801e39c26a38de78
  Stored in directory: /root/.cache/pip/wheels/b5

## upload

In [87]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data			   submit_sample.csv  train.csv
submission_LGB_LR_RF_ensemble.csv  test.csv


In [4]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

Saving submit_sample.csv to submit_sample.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


## Feature Engineering

In [192]:
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)

#ここから追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)
data["retirement"] = (data["age"] > 59).astype(int)


train["y"]=train["y"].astype(np.int64)

delete_columns = ['id', 'default', 'marital']
data=data.drop(delete_columns, axis=1)


categorical_features = ['job','education','housing','loan','contact','poutcome',
                        "income",'goodincome','married',"single",'housing+loan',
                        "retirement"]
data[categorical_features] = data[categorical_features].astype('category')

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.drop('y', axis = 1)



In [193]:
delete_columns

['id', 'default', 'marital']

In [194]:
data

,age,job,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,income,goodincome,married,single,year,campaign+previous,poutcome/pdays,housing+loan,duration*campaign,retirement
0,31,4,3,12294,1,0,2,21,1,101,3,498,0,0,0.0,0,0,1,1,51,3,0,1,303,0
1,29,7,1,43027,0,0,2,22,10,158,2,702,0,2,1.0,0,1,0,0,322,2,28,0,316,0
2,35,1,1,12252,1,0,2,11,1,351,1,826,0,1,0.0,0,1,1,1,41,1,11,1,351,0
3,31,2,3,99121,1,1,0,16,7,658,2,120,0,1,0.0,0,0,1,1,226,2,76,2,1316,0
4,48,10,0,42005,1,0,1,3,6,177,1,273,0,2,0.0,1,0,1,1,183,1,70,1,177,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,6,1,98357,1,0,2,6,9,101,2,417,0,1,NaN,0,1,1,1,276,2,23,1,202,0
18046,34,0,3,29621,1,0,2,12,7,345,1,815,0,2,NaN,0,0,1,1,222,1,24,1,345,0
18047,34,3,3,94260,1,0,0,16,7,121,2,370,0,2,NaN,0,1,0,0,226,2,52,1,242,0
18048,31,2,3,65483,1,0,0,15,7,345,2,41,0,2,NaN,0,0,0,0,225,2,392,1,690,0


# LightGBM

## def LGB

In [195]:
#LightGBMのデータ分割
X_train_LGB, X_test_LGB, y_train_LGB, y_test_LGB = train_test_split(X, y, test_size=0.2, random_state=0)
X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X, y, test_size=0.2, random_state=1)

In [196]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

## Optuna of LGB

In [197]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.15,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train_LGB, y_train_LGB)
    lgb_eval = lgb.Dataset(X_valid_LGB, y_valid_LGB, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid_LGB = model.predict(X_valid_LGB, num_iteration=model.best_iteration)
    score = log_loss(y_valid_LGB, y_pred_valid_LGB)
    return score

In [198]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=30)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.183185	valid_1's binary_logloss: 0.165843
[20]	training's binary_logloss: 0.152312	valid_1's binary_logloss: 0.141168
[30]	training's binary_logloss: 0.130338	valid_1's binary_logloss: 0.126298
[40]	training's binary_logloss: 0.113954	valid_1's binary_logloss: 0.113959
[50]	training's binary_logloss: 0.100857	valid_1's binary_logloss: 0.105269
[60]	training's binary_logloss: 0.0905326	valid_1's binary_logloss: 0.0984232
[70]	training's binary_logloss: 0.0813099	valid_1's binary_logloss: 0.0919385
[80]	training's binary_logloss: 0.0734049	valid_1's binary_logloss: 0.0864496
[90]	training's binary_logloss: 0.0663634	valid_1's binary_logloss: 0.0821787
[100]	training's binary_logloss: 0.0603149	valid_1's binary_logloss: 0.0781767
[110]	training's binary_logloss: 0.0541766	valid_1's binary_logloss: 0.0743709
[120]	training's binary_logloss: 0.0486519	valid_1's binary_logloss: 0.0700578
[130]	tra

[I 2020-08-24 12:42:22,616] Trial 0 finished with value: 0.048523823989085675 and parameters: {'max_bin': 427, 'num_leaves': 79}. Best is trial 0 with value: 0.048523823989085675.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.176956	valid_1's binary_logloss: 0.16222
[20]	training's binary_logloss: 0.144301	valid_1's binary_logloss: 0.137977
[30]	training's binary_logloss: 0.119363	valid_1's binary_logloss: 0.120048
[40]	training's binary_logloss: 0.101942	valid_1's binary_logloss: 0.107525
[50]	training's binary_logloss: 0.0882003	valid_1's binary_logloss: 0.0982029
[60]	training's binary_logloss: 0.0775942	valid_1's binary_logloss: 0.090586
[70]	training's binary_logloss: 0.0682231	valid_1's binary_logloss: 0.0845657
[80]	training's binary_logloss: 0.0605564	valid_1's binary_logloss: 0.0796208
[90]	training's binary_logloss: 0.0532918	valid_1's binary_logloss: 0.0741255
[100]	training's binary_logloss: 0.0481037	valid_1's binary_logloss: 0.0706625
[110]	training's binary_logloss: 0.0429041	valid_1's binary_logloss: 0.0670822
[120]	training's binary_logloss: 0.0385013	valid_1's binary_logloss: 0.0642672
[130]	tra

[I 2020-08-24 12:42:25,472] Trial 1 finished with value: 0.04910344546188399 and parameters: {'max_bin': 372, 'num_leaves': 96}. Best is trial 0 with value: 0.048523823989085675.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.17688	valid_1's binary_logloss: 0.162437
[20]	training's binary_logloss: 0.143662	valid_1's binary_logloss: 0.136135
[30]	training's binary_logloss: 0.119455	valid_1's binary_logloss: 0.118749
[40]	training's binary_logloss: 0.102071	valid_1's binary_logloss: 0.106072
[50]	training's binary_logloss: 0.0883508	valid_1's binary_logloss: 0.0967712
[60]	training's binary_logloss: 0.0771405	valid_1's binary_logloss: 0.0888738
[70]	training's binary_logloss: 0.0683603	valid_1's binary_logloss: 0.0832056
[80]	training's binary_logloss: 0.0595594	valid_1's binary_logloss: 0.0772371
[90]	training's binary_logloss: 0.0525642	valid_1's binary_logloss: 0.0722784
[100]	training's binary_logloss: 0.0469586	valid_1's binary_logloss: 0.068936
[110]	training's binary_logloss: 0.042054	valid_1's binary_logloss: 0.0655499
[120]	training's binary_logloss: 0.0373672	valid_1's binary_logloss: 0.0625969
[130]	trai

...

[I 2020-08-24 12:43:49,569] Trial 27 finished with value: 0.04829832799621358 and parameters: {'max_bin': 499, 'num_leaves': 55}. Best is trial 16 with value: 0.04654357980084098.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.174269	valid_1's binary_logloss: 0.160609
[20]	training's binary_logloss: 0.139108	valid_1's binary_logloss: 0.132779
[30]	training's binary_logloss: 0.114712	valid_1's binary_logloss: 0.115772
[40]	training's binary_logloss: 0.0964486	valid_1's binary_logloss: 0.102485
[50]	training's binary_logloss: 0.0828443	valid_1's binary_logloss: 0.0934372
[60]	training's binary_logloss: 0.0715018	valid_1's binary_logloss: 0.0859351
[70]	training's binary_logloss: 0.0622762	valid_1's binary_logloss: 0.0798645
[80]	training's binary_logloss: 0.0546876	valid_1's binary_logloss: 0.0751978
[90]	training's binary_logloss: 0.0479526	valid_1's binary_logloss: 0.0708187
[100]	training's binary_logloss: 0.0416363	valid_1's binary_logloss: 0.0668695
[110]	training's binary_logloss: 0.0366277	valid_1's binary_logloss: 0.0638737
[120]	training's binary_logloss: 0.0324353	valid_1's binary_logloss: 0.0607425
[130]	

[I 2020-08-24 12:43:52,482] Trial 28 finished with value: 0.04869251028884313 and parameters: {'max_bin': 418, 'num_leaves': 107}. Best is trial 16 with value: 0.04654357980084098.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.179773	valid_1's binary_logloss: 0.164187
[20]	training's binary_logloss: 0.148169	valid_1's binary_logloss: 0.139654
[30]	training's binary_logloss: 0.125695	valid_1's binary_logloss: 0.12356
[40]	training's binary_logloss: 0.108849	valid_1's binary_logloss: 0.111527
[50]	training's binary_logloss: 0.0961821	valid_1's binary_logloss: 0.103041
[60]	training's binary_logloss: 0.0861355	valid_1's binary_logloss: 0.0958722
[70]	training's binary_logloss: 0.0768487	valid_1's binary_logloss: 0.0895626
[80]	training's binary_logloss: 0.0697411	valid_1's binary_logloss: 0.0849575
[90]	training's binary_logloss: 0.0628254	valid_1's binary_logloss: 0.0798361
[100]	training's binary_logloss: 0.0561203	valid_1's binary_logloss: 0.0757668
[110]	training's binary_logloss: 0.0508616	valid_1's binary_logloss: 0.0728779
[120]	training's binary_logloss: 0.046215	valid_1's binary_logloss: 0.069845
[130]	train

[I 2020-08-24 12:43:55,758] Trial 29 finished with value: 0.04952741524108397 and parameters: {'max_bin': 369, 'num_leaves': 87}. Best is trial 16 with value: 0.04654357980084098.


In [199]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': study.best_params["num_leaves"],
    'boosting_type' : 'gbdt',
    'objective': 'binary',
    'max_bin': study.best_params["max_bin"],
    'max_dapth': 7
}



##Bagging of LGB

In [200]:
def bagging(seed):
    positive_count_train = y_train_LGB.sum()
    sampler = RandomUnderSampler(sampling_strategy={0:int(positive_count_train), 1:int(positive_count_train)},random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train_LGB, y_train_LGB)
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=seed)
    model_bagging = lgbm_train(X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB, lgbm_params)
    return model_bagging

In [201]:
%%time
models_LGB = []
for i in range(10):
    models_LGB.append(bagging(i))



[1]	valid_0's binary_logloss: 0.677199
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.661076
[3]	valid_0's binary_logloss: 0.645925
[4]	valid_0's binary_logloss: 0.633197
[5]	valid_0's binary_logloss: 0.620732
[6]	valid_0's binary_logloss: 0.609193
[7]	valid_0's binary_logloss: 0.598475
[8]	valid_0's binary_logloss: 0.589049
[9]	valid_0's binary_logloss: 0.580219
[10]	valid_0's binary_logloss: 0.570882
[11]	valid_0's binary_logloss: 0.563916
[12]	valid_0's binary_logloss: 0.556798
[13]	valid_0's binary_logloss: 0.550563
[14]	valid_0's binary_logloss: 0.54511
[15]	valid_0's binary_logloss: 0.539096
[16]	valid_0's binary_logloss: 0.534771
[17]	valid_0's binary_logloss: 0.530005
[18]	valid_0's binary_logloss: 0.524141
[19]	valid_0's binary_logloss: 0.519466
[20]	valid_0's binary_logloss: 0.514915
[21]	valid_0's binary_logloss: 0.509528
[22]	valid_0's binary_logloss: 0.504703
[23]	valid_0's binary_logloss: 0.501275
[24]	valid_0's binary_loglos

In [202]:
y_preds_LGB = []

for m in models_LGB:
    y_preds_LGB.append(m.predict(X_test_LGB, num_iteration=m.best_iteration))

y_preds_bagging_LGB = sum(y_preds_LGB)/len(y_preds_LGB)
# auc を計算する
auc = roc_auc_score(y_test_LGB, y_preds_bagging_LGB)
print(auc)

y_subs_LGB = []
for m in models_LGB:
    y_subs_LGB.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging_LGB = sum(y_subs_LGB)/len(y_subs_LGB)
# auc を計算する
y_subs_bagging_LGB

0.8655835675261715


array([0.97434199, 0.31304721, 0.1020039 , ..., 0.23753478, 0.01384158,
       0.53885567])

In [203]:
percentage_LGB=0.23
y_sub_LGB = (y_subs_bagging_LGB > percentage_LGB).astype(int)

# sub['B'] = y_sub_LGB
# sub.to_csv('submission_lightgbm_bagging.csv', index=False, header=False)

sum(y_sub_LGB),len(y_sub_LGB)

(9058, 18050)

##Holdout of LGB

In [204]:
#LightGBM_Hのデータ分割
X_train_LGB, X_test_LGB, y_train_LGB, y_test_LGB = train_test_split(X, y, test_size=0.3, random_state=7)
X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X, y, test_size=0.3, random_state=8)

In [205]:
def holdout(seed):
    positive_count_train = y_train_LGB.sum()
    sampler = RandomUnderSampler(sampling_strategy={0:int(positive_count_train)*2, 1:int(positive_count_train)},random_state=seed+20, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train_LGB, y_train_LGB)
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=seed)
    model_holdout = lgbm_train(X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB, lgbm_params)
    return model_holdout

In [206]:

y_pred_H = holdout(18).predict(X_test_LGB, num_iteration=holdout(18).best_iteration)
y_preds_bagging_LGB = sum(y_pred_H)/len(y_pred_H)
# auc を計算する
auc = roc_auc_score(y_test_LGB, y_pred_H)


y_subs_H_LGB = holdout(18).predict(X_sub, num_iteration=holdout(18).best_iteration)

y_subs_H_LGB2 = sum(y_subs_H_LGB)/len(y_subs_H_LGB)
# auc を計算する
y_subs_H_LGB2

[1]	valid_0's binary_logloss: 0.619005
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.602224
[3]	valid_0's binary_logloss: 0.587648
[4]	valid_0's binary_logloss: 0.573756
[5]	valid_0's binary_logloss: 0.561234
[6]	valid_0's binary_logloss: 0.550234
[7]	valid_0's binary_logloss: 0.539704
[8]	valid_0's binary_logloss: 0.52993
[9]	valid_0's binary_logloss: 0.52216
[10]	valid_0's binary_logloss: 0.51332
[11]	valid_0's binary_logloss: 0.505759
[12]	valid_0's binary_logloss: 0.498712
[13]	valid_0's binary_logloss: 0.4919
[14]	valid_0's binary_logloss: 0.485855
[15]	valid_0's binary_logloss: 0.479855
[16]	valid_0's binary_logloss: 0.475015
[17]	valid_0's binary_logloss: 0.470394
[18]	valid_0's binary_logloss: 0.465196
[19]	valid_0's binary_logloss: 0.459927
[20]	valid_0's binary_logloss: 0.455397
[21]	valid_0's binary_logloss: 0.450699
[22]	valid_0's binary_logloss: 0.445916
[23]	valid_0's binary_logloss: 0.442506
[24]	valid_0's binary_logloss: 0

0.1954011334540482

In [207]:
percentage_H=0.1
y_sub_H = (y_subs_H_LGB > percentage_H).astype(int)

# sub['B'] = y_sub_LGB
# sub.to_csv('submission_lightgbm_bagging.csv', index=False, header=False)

sum(y_sub_H),len(y_sub_H)

(8341, 18050)

# RandomForest


## def RF

In [208]:
#RandomForestのデータ分割
X_train_RF, X_test_RF, y_train_RF, y_test_RF = train_test_split(X, y, test_size=0.2, random_state=12)
#X_train2, X_valid, y_train2, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [209]:
def RF(seed):
    positive_count_train = y_train_RF.sum()
    sampler_RF = RandomUnderSampler(sampling_strategy={0:int(positive_count_train), 1:int(positive_count_train)},random_state=seed+10, replacement=True)
    X_resampled_RF, y_resampled_RF = sampler_RF.fit_resample(X_train_RF, y_train_RF)
    clf = RandomForestClassifier(n_estimators=100,max_depth=4, random_state=seed)
    clf.fit(X_resampled_RF, y_resampled_RF)

    return clf

## Bagging of RF

In [210]:
%%time
models_RF = []

for i in range(10):
    models_RF.append(RF(i))

CPU times: user 3.09 s, sys: 22.3 ms, total: 3.11 s
Wall time: 3.12 s


In [211]:
y_preds_RF = []
percentage_RF=0.004

for clf in models_RF:
    y_preds_RF.append(clf.predict(X_sub))
y_subs_bagging_RF = sum(y_preds_RF)/len(y_preds_RF)
y_sub_RF = (y_subs_bagging_RF > percentage_RF).astype(int)

sum(y_sub_RF),len(y_sub_RF)

(7879, 18050)

In [212]:
y_sub_LGB

array([1, 1, 0, ..., 1, 0, 1])

## 全部　RF

In [213]:
"""
RF_clf = RandomForestClassifier(n_estimators=100,max_depth=30, random_state=13)
RF_clf.fit(X, y)
y_pred_RF=RF_clf.predict(X_sub)
percentage_RF=0.004
y_sub_RF = (y_pred_RF > percentage_RF).astype(int)

sum(y_sub_RF),len(y_sub_RF)
"""

'\nRF_clf = RandomForestClassifier(n_estimators=100,max_depth=30, random_state=13)\nRF_clf.fit(X, y)\ny_pred_RF=RF_clf.predict(X_sub)\npercentage_RF=0.004\ny_sub_RF = (y_pred_RF > percentage_RF).astype(int)\n\nsum(y_sub_RF),len(y_sub_RF)\n'

# Ensemble

In [214]:
sub2 = pd.DataFrame({'y_sub_LGB': y_sub_LGB,
                     'y_sub_LR': y_sub_LR,
                     'y_sub_RF': y_sub_RF,
                     'y_sub_H': y_sub_H
                     })
sub2.corr()

,y_sub_LGB,y_sub_LR,y_sub_RF,y_sub_H
y_sub_LGB,1.000000,0.133561,0.659100,0.667687
y_sub_LR,0.133561,1.000000,0.153279,0.149122
y_sub_RF,0.659100,0.153279,1.000000,0.605649
y_sub_H,0.667687,0.149122,0.605649,1.000000


In [215]:
sum(y_sub_LGB),sum(y_sub_LR),sum(y_sub_RF),sum(y_sub_H)

(9058, 9769, 7879, 8341)

In [216]:
sum_subs = y_sub_LGB + y_sub_H + y_sub_RF
#sub['B'] = (sum_subs >= 2).astype(int)
sub['B'] = y_sub_LGB

sub.to_csv('submission_LGB_H_RF_ensemble.csv', index=False, header=False)
sub.head()

,A,B
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


In [217]:
sum(sub["B"])

9058

# Output

In [224]:
files.download('submission_LGB_H_RF_ensemble.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LogisticRegression


## def LogiReg

In [219]:
#LogisticRegressionのデータ分割
X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split(X, y, test_size=0.2, random_state=11)
#X_train2, X_valid, y_train2, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [220]:
def LogiReg(seed):
    positive_count_train = y_train_LR.sum()
    sampler_LR = RandomUnderSampler(sampling_strategy={0:int(positive_count_train), 1:int(positive_count_train)},random_state=seed, replacement=True)
    X_resampled_LR, y_resampled_LR = sampler_LR.fit_resample(X_train_LR, y_train_LR)
    clf = LogisticRegression(penalty='l2', solver="sag", random_state=seed)
    clf.fit(X_resampled_LR, y_resampled_LR)

    return clf

## Bagging of LogiReg

In [221]:
%%time
models_LR = []

for i in range(10,20):
    models_LR.append(LogiReg(i))

CPU times: user 1.31 s, sys: 9.13 ms, total: 1.31 s
Wall time: 1.32 s


In [222]:
y_preds_LR = []
percentage_LR=0.2

for clf in models_LR:
    y_preds_LR.append(clf.predict(X_sub))
y_subs_bagging_LR = sum(y_preds_LR)/len(y_preds_LR)
y_sub_LR = (y_subs_bagging_LR > percentage_LR).astype(int)

sum(y_sub_LR),len(y_sub_LR)

(9769, 18050)

## 全部　LR

In [223]:
"""
LR_clf = LogisticRegression(penalty='l2', solver="sag", random_state=15)
LR_clf.fit(X, y)
y_pred_LR=LR_clf.predict(X_sub)
percentage_LR=0.4
y_sub_LR = (y_pred_LR > percentage_LR).astype(int)

sum(y_sub_LR),len(y_sub_LR)
"""

'\nLR_clf = LogisticRegression(penalty=\'l2\', solver="sag", random_state=15)\nLR_clf.fit(X, y)\ny_pred_LR=LR_clf.predict(X_sub)\npercentage_LR=0.4\ny_sub_LR = (y_pred_LR > percentage_LR).astype(int)\n\nsum(y_sub_LR),len(y_sub_LR)\n'